# Automating Job Applications

I spend >60 minutes daily to applying for Jobs.

This is the strategy:
Be among the first ones to apply to jobs. That way, I score an interview.

Most of my applications are numerous to Chip Companies.

### Step 1: Scouting Relevant roles on Chip Company Websites

I browse roles opened up in the last 24 hours, every day. I open the relevant job positions in new browser tabs. 

I have the job portals of chip and compiler companies open in a specific tab group

eg. Chip companies: Intel Amd Apple Nvidia Qualcomm Broadcomm ARM Siemens Micron ADI Groq d-Matrix OctoML Meta Amazon Google Modular etc

Occasionally, I peek over the description to see how relevant it is to my profile

### Step 2: Applying to those roles

This is the most time consuming part. Routine Form Filling for most of the portals. Unless I apply via Linkedin.com/jobs

Go to a role tab, fill out the forms, use past application, many mouse clicks later, I hit submit

How about I automate step 2. with Selenium

In [ ]:
import subprocess
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException


In [ ]:
# Get browser tab urls, ability to change tab, close tab
# I will have to filter by Job application pages

def getSafariURLs():
    allTabs = '''
tell application "Safari"
    set allTabUrls to ""
    set windowCount to count of windows
    
    repeat with i from 1 to windowCount
        set tabCount to count of tabs of window i
        
        repeat with j from 1 to tabCount
            set tabUrl to URL of tab j of window i
            set allTabUrls to allTabUrls & tabUrl & linefeed
        end repeat
    end repeat
    
    return allTabUrls
end tell
'''

    res = subprocess.run(['osascript', '-e', allTabs], capture_output=True, text=True)
    arr = res.stdout.split()
    return arr

def closeTab():
    closeTab = '''
    tell application "Safari"
        tell window 1
            close current tab
        end tell
    end tell
    '''
    subprocess.run(['osascript', '-e', closeTab], capture_output=True, text=True)
    return getSafariURLs()

def nextTab():
    nextTab = '''
    tell application "Safari"
        tell window 1
            set currentTabIndex to index of current tab
            set nextTabIndex to currentTabIndex + 1
            if nextTabIndex is not greater than count of tabs then
                set current tab to tab nextTabIndex
            end if
        end tell
    end tell
    '''
    subprocess.run(['osascript', '-e', nextTab], capture_output=True, text=True)
    return getSafariURLs()

In [ ]:
# Let's filter for Apple and Intel tabs

job_url = getSafariURLs()
apple = []
intel = []
for url in job_url:
  if "jobs.apple.com/en-us/details" in url:
    apple += [url]
  elif "https://jobs.intel.com/en/job/" in url or "https://intel.wd1.myworkdayjobs.com/en-US/External/job/" in url:
    intel += [url]
intel

In [ ]:
# Lets use Firefox, so, I can login manually to the job portals.

driver = webdriver.Firefox()
driver.maximize_window()

In [ ]:
# WebDriverWait has a clunky api. I can use a wrapper function

def delayedFind(searchBy, value, patience=5) :
  return WebDriverWait(driver,patience).until(EC.visibility_of_element_located((searchBy, value)))


In [ ]:

# Sign in all job portals
def appleSignIn():
  driver.get("https://jobs.apple.com/en-us/login")
  input("Please login to Apple.com")

def intelSignIn():
  driver.get("https://intel.wd1.myworkdayjobs.com/en-US/External")
  delayedFind(By.CLASS_NAME, '''css-1xtbc5b''').click()
  INTEL_LOGIN = input("Intel ID")
  delayedFind(By.ID, "input-11").send_keys(INTEL_LOGIN)
  INTEL_PASSWORD = input("Intel Password")
  delayedFind(By.ID, "input-12").send_keys(INTEL_PASSWORD)
  delayedFind(By.CLASS_NAME, "css-maqlan").click()
  input("Please login to Intel.com")
  
# appleSignIn()
# intelSignIn()


In [ ]:
def apple_Submit(url):
  driver.get(url)
  try:
    driver.find_element(By.PARTIAL_LINK_TEXT, "Submit Resume").click()
    continueBtn = WebDriverWait(driver,5).until(EC.visibility_of_element_located((By.ID, "applyResumeContinue")))
    continueBtn.click()
    submitBtn = WebDriverWait(driver,5).until(EC.visibility_of_element_located((By.ID, "applyReviewSubmit")))
    submitBtn.click()
  except Exception as e:
    print(e)

for appleJob in apple:
  apple_Submit(appleJob)

In [ ]:
# Let's apply to Intel's job portal
def intel_Submit(url):
  print(url)
  driver.get(url)
  delayedFind( By.LINK_TEXT, "Apply").click()
  delayedFind(By.LINK_TEXT, "Use My Last Application").click()
  delayedFind(By.ID, "input-1", 15).click()
  input("Select one")
  # Intel's How did you find this Job section: Difficulty to do with Selenium

for intelJob in intel:
  intel_Submit(intelJob)
